[Reference](https://ai.gopubby.com/how-i-used-a-u-net-to-map-building-footprints-from-the-sky-bf6d184c41d8)

# Encoder Block

In [1]:
def encoder_block(filters, inputs, dropout_rate=0.1, kernel_initializer='lecun_normal'):
    x = Conv2D(filters, kernel_size=(3, 3), padding='same', strides=1, activation='relu',
               kernel_initializer=kernel_initializer)(inputs)
    x = SpatialDropout2D(dropout_rate)(x)
    s = Conv2D(filters, kernel_size=(3, 3), padding='same', strides=1, activation='relu',
               kernel_initializer=kernel_initializer)(x)
    s = SpatialDropout2D(dropout_rate)(s)
    p = MaxPooling2D(pool_size=(2, 2), padding='same')(s)
    return s, p

# Bottleneck Layer

In [2]:
def baseline_layer(filters, inputs, dropout_rate=0.1, kernel_initializer='lecun_normal'):
    x = Conv2D(filters, kernel_size=(3, 3), padding='same', strides=1, activation='relu',
               kernel_initializer=kernel_initializer)(inputs)
    x = SpatialDropout2D(dropout_rate)(x)
    x = Conv2D(filters, kernel_size=(3, 3), padding='same', strides=1, activation='relu',
               kernel_initializer=kernel_initializer)(x)
    x = SpatialDropout2D(dropout_rate)(x)
    return x

# Decoder Block

In [3]:
def decoder_block(filters, connections, inputs, dropout_rate=0.1, kernel_initializer='lecun_normal'):
    x = Conv2DTranspose(filters, kernel_size=(2, 2), padding='same', activation='relu', strides=2,
                        kernel_initializer=kernel_initializer)(inputs)
    skip_connections = concatenate([x, connections], axis=-1)
    x = Conv2D(filters, kernel_size=(3, 3), padding='same', activation='relu',
               kernel_initializer=kernel_initializer)(skip_connections)
    x = SpatialDropout2D(dropout_rate)(x)
    x = Conv2D(filters, kernel_size=(3, 3), padding='same', activation='relu',
               kernel_initializer=kernel_initializer)(x)
    x = SpatialDropout2D(dropout_rate)(x)
    return x

# Final Output Layer

In [4]:
# outputs = Conv2D(1, 1, activation = 'sigmoid')(d4)

# Putting all of the Pieces Together

In [5]:
def unet():
  inputs = Input(shape = (256, 256, 3)) #defines the input layer and shape of images

  #encoder
  s1, p1 = encoder_block(32, inputs = inputs)
  s2, p2 = encoder_block(64, inputs = p1)
  s3, p3 = encoder_block(128, inputs = p2)
  s4, p4 = encoder_block(256, inputs = p3)

  #bottleneck
  baseline = baseline_layer(512, p4)

  #decoder
  d1 = decoder_block(256, s4, baseline)
  d2 = decoder_block(128, s3, d1)
  d3 = decoder_block(64, s2, d2)
  d4 = decoder_block(32, s1, d3)

  #output function for binary classification of pixels
  outputs = Conv2D(1, 1, activation = 'sigmoid')(d4)

  #finalizing the model
  model = Model(inputs = inputs, outputs = outputs, name = 'Unet')

  return model